# 네이버 신용카드 크롤링

## 1. 조건을 계속 다르게 하면서 10개 미만씩 긁어 모으기
카드를 선택하는 과정에서 조건을 추가하면 10개 미만으로 카드들이 나오게 할 수 있습니다.
이것을 반복하여 긁는 방법인데, 결론부터 말씀드리자면 364개중에 361개만 긁기 성공하였습니다.

In [3]:
import requests, time, datetime
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

def my_random_choice1(l,s=0,e=5):
    n = np.random.choice(range(s,e),1)[0]
    return ','.join(np.random.choice(l,n))
def my_random_choice2(l,s=0,e=5):
    n = np.random.choice(range(s,e),1,[0.4,0.3,0.1,0.1,0.1])[0]
    return ','.join(np.random.choice(l,n))
def extract_card_name(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('li', class_='item')
    names = [i.find('b', class_='name').text for i in items]
    image_info = [i.find('img',class_='img')['src'] for i in items]
    
    return [(n,i) for n,i in zip(names,image_info)]

company_Code = ['SH','HD','SS','KB','LO','SK','WR','CT','NH','IB']
brand_Names = ['mastercard','visa','unionpay','jcb','bc','amex']
benefit_Category_Ids = [1,12,7,16,14,13,17,15,8,10,9,11,18,19,4,5,6,2,3,21,20,22,52,62,63,83,103,123,143,163,1001,1002,1003]
benefit_Category_Ids = [str(i) for i in benefit_Category_Ids]
annualFee = ['','&maxAnnualFee=10000','&maxAnnualFee=30000','&maxAnnualFee=100000','&minAnnualFee=100000']
sortMethod = ['&sortMethod=qc','&sortMethod=ri']

In [5]:
start = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
total_num = 364
name_list = []
n = len(name_list)
root_url = 'https://card-search.naver.com/list?'
end_url = '&query=%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C%EC%B6%94%EC%B2%9C&isRefetch=true'
i = 0
while (total_num - n) > 0:
    
    CC = my_random_choice2(company_Code) #company_Code
    BN = my_random_choice1(brand_Names) #brand_Names
    BCI = my_random_choice1(benefit_Category_Ids) #benefit_Category_Ids
    AF = np.random.choice(annualFee,1)[0] # annual_fee
    SM = np.random.choice(sortMethod,1)[0] # sort method
    content_url = f'companyCode={CC}&brandNames={BN}&benefitCategoryIds={BCI}{AF}{SM}'
    temp_url = root_url + content_url + end_url
    temp_name = extract_card_name(temp_url)
    
    name_list.extend(temp_name)
    name_list = list(set(name_list))
    n = len(name_list)
    
    i += 1
    time.sleep(0.1)
#     if len(temp_num)>=3:
#         print(temp_url)
    print(f'현재 가져온 카드 갯수 : {n}/{total_num}, 시도한 크롤링 횟수 : {i},   시작시간 : {start}', end='\r')
# 1시간정도 돌리면 360개정도는 모아집니다.
# case나눠주는거를 10개 미만으로 나누어지게 disjoint하게 다 돌릴 수 있으면 좋을 것 같은데 방법을 아직 찾지 못했습니다.

KeyboardInterrupt: 

In [ ]:
card_name = [a[0] for a in name_list]
card_tag = [a[1].split('/')[5] for a in name_list]

AMF_list, CH_list, BE_list = [], [], []

start_url = 'https://card-search.naver.com/item?'
end_url = '&pquery=%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C%EC%B6%94%EC%B2%9C'
for i in range(len(card_name)):
    content_url = f'cardAdId={card_tag[i]}&query={card_name[i]}'
    temp_url = start_url + content_url + end_url
    response = requests.get(temp_url)

    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
#     print(f'카드이름 : {card_name[i]}, url : {temp_url}')

    # AMF: annual membership fee, CH: credit history, BE: benefit
    box = soup.find('dl', class_='list')
    AMF = box.find('dd',class_='desc as_annualFee').text
    CH = ''
    BE = ''
    
    AMF_list.append(AMF)
    
    time.sleep(0.1)
    print(f'카드 정보 긁어오는중...  {i}/{len(card_name)-1}', end='\r')

In [ ]:

df = pd.DataFrame({'카드이름':card_name,
              '연회비':AMF_list,
              '기준실적':'',
              '혜택요약':''})
df

,카드이름,연회비,기준실적,혜택요약
0,LOCA CLASSIC (로카 클래식),"국내 2만원,해외 2만원",,
1,롯데백화점 FLEX카드,"국내 10만원,해외 10만원",,
2,참! 좋은 친구 청년동행카드,"국내 2천원,해외 5천원",,
3,위메프우리카드,"국내 1만원,해외 1만2천원신규 최대 1만2천원 지급",,
4,신한 Hi-Point 카드,해외 8천원신규 최대 1만5천원 지급,,
...,...,...,...,...
263,카드의정석 댕댕냥이,"국내 1만원,해외 1만2천원신규 최대 1만2천원 지급",,
264,1Q Special+ (원큐 스페셜플러스),해외 1만원,,
265,KB국민 굿데이올림카드,"국내 1만5천원,해외 2만원신규 최대 1만5천원 지급",,
266,Smart Any (스마트애니),"국내 5천원,해외 5천원신규 최대 5천원 지급",,


## 2. github에 올려둔 뒤 html정보 가져와서 사용하기
미리 364개를 긁은 html을 github에 업로드 해둔 뒤 해당 html을 가져와서 사용합니다. 서버에서도 아마 되지 않을까 싶습니다. 이러면 정말 쉽게 가능합니다..!!

In [31]:
import urllib
target_url = 'https://raw.githubusercontent.com/dykim-stat/for_share/main/naver_credit_card_html.txt'
file = urllib.request.urlopen(target_url)

html = ''
for line in file:
    decoded_line = line.decode("utf-8")
    html += decoded_line

In [37]:
soup = BeautifulSoup(html, 'html.parser')
items = soup.find_all('li', class_='item')
card_name = [i.find('b', class_='name').text for i in items]
image_info = [i.find('img',class_='img')['src'] for i in items]
card_tag = [a.split('/')[5] for a in image_info]

In [102]:
NTC_list, AMF_list, CH_list, BE_list = [], [], [], []

start_url = 'https://card-search.naver.com/item?'
end_url = '&pquery=%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C%EC%B6%94%EC%B2%9C'
for i in range(len(card_name)):
# for i in range(10):
    content_url = f'cardAdId={card_tag[i]}&query={card_name[i]}'
    temp_url = start_url + content_url + end_url
    response = requests.get(temp_url)

    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
#     print(f'카드이름 : {card_name[i]}, url : {temp_url}')
    try:
        title_src = soup.find('dl', class_='list').find_all('dt', class_='title')
        content_src = soup.find('dl', class_='list').find_all('dd', class_='desc')
        title = [t.text for t in title_src]
        content = [c.text for c in content_src]
        result_dict = dict((w,c) for w,c in zip(title,content))
    except AttributeError:
        print(f'카드정보가 사라진 카드 존재, 카드이름 {card_name[i]}')
        
    ## NTC: Notice, AMF: annual membership fee, CH: credit history, BE: benefit
    loop_list = [('공지','NTC'), ('연회비','AMF'), ('기준실적','CH'), ('혜택요약','BE')]
    for j,k in loop_list:
        try:
            globals()[k] = result_dict[j]
        except KeyError:
            globals()[k] = ' '

    
    NTC_list.append(NTC)
    AMF_list.append(AMF)
    CH_list.append(CH)
    BE_list.append(BE)
    time.sleep(0.1)
    print(f'카드 정보 긁어오는중...  {i+1}/{len(card_name)}', end='\r')

카드정보가 사라진 카드 존재, 카드이름 신한카드 Deep Oil


In [103]:

df = pd.DataFrame({'카드이름':card_name,
              '공지':NTC_list,
              '연회비':AMF_list,
              '기준실적':CH_list,
              '혜택요약':BE_list})
df

,카드이름,공지,연회비,기준실적,혜택요약
0,삼성카드 taptap O,연 최대 36만원 할인 혜택!,"국내 1만원,해외 1만원신규 최대 1만원 지급",직전 1개월 합계 30만원 이상,"스타벅스50%, 교통·통신10% 할인G마켓·옥션·11번가 7% 할인"
1,KB국민 청춘대로 톡톡카드,스타벅스 최대 60% 할인,"국내 1만원,해외 1만2천원신규 최대 1만2천원 지급",직전 1개월 합계 30만원 이상,"스벅 최대60%, 교통·통신10%할인20년도 KB국민카드 온라인발급 1위"
2,신한카드 Mr.Life,365일 야간에도 주말에도 빠르게!,"국내 1만5천원,해외 1만8천원신규 최대 1만8천원 지급",직전 1개월 합계 30만원 이상,"생활혜택 연간 최대 60만원 할인!쇼핑,통신,마트,외식 주요혜택 10%"
3,삼성카드 4,모든 가맹점 최대 1% 결제일할인,"국내 5천원,해외 1만원신규 최대 1만원 지급",조건없음,실적한도 제한없이 어디서나 할인!모든 가맹점에서 2~3개월 무이자할부
4,LIKIT ALL 카드 (라이킷 올),연간 최대 24만원 할인 혜택!,"국내 1만원,해외 1만원신규 최대 1만원 지급",조건없음,"실적조건 없이 모든 가맹점 1% 할인주유소,편의점 건당 최대 5천원 할인"
...,...,...,...,...,...
359,카드의정석 시니어플러스,,"국내 1만원,해외 1만2천원",직전 1개월 합계 30만원 이상,"모아포인트 최대 3% 적립공항 라운지, 영화, 커피 할인"
360,위메프페이 신한카드,365일 야간에도 주말에도 빠르게!,"국내 1만2천원,해외 1만5천원",조건없음,"간편 등록, 편리한 사용 위메프페이!기본 0.7% 위메프 2% 특별적립!"
361,하나투어 KB국민카드,"인터파크 영화 1매당 3,500원할인","국내 1만원,해외 1만원",직전 3개월 합계 30만원 이상,하나투어 마일리지 최대 10% 적립하나투어 마일리지클럽 서비스 제공
362,KB국민 우체국 스마트 카드,,"국내 1만1천원,해외 1만5천원",직전 1개월 합계 30만원 이상,우체국알뜰폰통신료 6~15천원청구할인우체국 우편료/쇼핑 10% 청구할인
